# Getting Started

In this tutorial, we'll learn how to simulate probabilities with ``pathfinder2e_stats``.

To follow this tutorial, you'll need to have at least a basic understanding of
- The [Pathfinder rules](https://2e.aonprd.com/PlayersGuide.aspx), and
- Data science workflows, e.g. based on Python + pandas + Jupyter notebooks. See {ref}`audience`.

If you don't have your Jupyter Notebook development environment ready yet, go back to {doc}`../installing`.

## Rolling some dice
Let's start simple - let's import the module and roll a d6.

We're going to roll it *one hundred thousand times.*

In [ ]:
import pathfinder2e_stats as pf2

oned6 = pf2.roll(1, 6)
oned6

``pathfinder2e_stats`` functions return standard {class}`xarray.DataArray` and {class}`xarray.Dataset` objects, which can be analyzed with standard data science techniques. We can start immediately answering some questions - for example, what is the mean roll?

In [ ]:
oned6.mean()

Note that the result above is a *numerical approximation*: the mean of rolling 1d6 an *infinite* amount of times is *exactly* 3.5. If we roll it less times than that, however, there's going to be some error.

If we roll it again, we are going to get a different sequence. This is because ``pathfinder2e_stats`` uses a global random number generator, which by default is reset to a fixed seed every time you restart your notebook. See {func}`~pathfinder2e_stats.seed`.

In [ ]:
pf2.roll(1, 6)

For the sake of convenience, we can quickly visualize arbitrary DataArrays and Dataset with the custom accessor `.display()`, available for all xarray objects after importing `pathfinder2e_stats`:

In [ ]:
pf2.roll(1, 6).display("1d6")

Well, that was easy, but we could have figured out the answer by doing the maths on the back of an envelope! Let's move on to something that is more complicated. A timeless classic: a 6d6 {prd_spells}`Fireball <1530>`!

In [ ]:
fireball = pf2.roll(6, 6)
fireball

What is the damage distribution? First we're going to calculate it numerically; then we'll visualize it with ``matplotlib`` (but we could use any other library, like ``plotly`` or ``hvplot``).

In [ ]:
fireball.value_counts("roll").to_pandas().to_frame("count")

In [ ]:
_ = fireball.to_pandas().hist(bins=30)

But wait - that's just the base damage! The *actual* damage of a fireball depends on the target's reflex saving throw, as well as their resistances, immunities and weaknesses. ``pathfinder2e_stats`` makes dealing with all this very easy.

## Rolling checks

In Pathfinder, a *check* is whenever one rolls a d20+bonus against a DC; this includes attack rolls against AC.

For example, a paladin with +8 Diplomacy tries to convince a guard to let them pass. The DC is 15.
To simulate that, we call {func}`~pathfinder2e_stats.check`.

In [ ]:
request = pf2.check(8, DC=15)
request

The output of {func}`~pathfinder2e_stats.check` is a Dataset, which contains several variables. We normally only care about the last one, `outcome`. However, there are several other variables before it that explain *how* we reached that outcome, allowing us to fully trace its logic:

- **natural** is the bare d20 roll, 100,000 times
- **outcome** is the roll's degree of success, taking into account critical success/failure rules, natural 1s and 20s.

`outcome` is an integer (sadly there are no categorical dtypes in xarray yet), whose meaning is mapped in the `legend` attribute of the dataset, as shown above. It is also available in the {class}`~pathfinder2e_stats.DoS` enum. For the sake of robustness and readability, when you express an outcome (we'll see later when and how) you should always use `DoS` and never its numerical value.

Have a look at the {func}`API documentation <pathfinder2e_stats.check>` for additional parameters, such as fortune/misfortune effects to roll twice and take highest/lowest, conditionally using hero points depending on initial outcome and special rules like the {prd_equipment}`Keen <2843>` rune.

You can aggregate the result by using handy helpers such as {func}`~pathfinder2e_stats.outcome_counts`:

In [ ]:
# Probability to get each outcome
pf2.outcome_counts(request).to_pandas().to_frame("%") * 100.0

It's also common to compare against `DoS`. Operators `>`, `>=`, `<`, and `<=` are supported:

In [ ]:
# Probability to get at least a success
(request.outcome >= pf2.DoS.success).mean().item()

In [ ]:
# Probability to get a critical success
(request.outcome == pf2.DoS.critical_success).mean().item()

Attack rolls, saving throws, counteract checks, flat checks, etc. work exactly in the same way as skill checks.
For example, the party rogue can Strike a bandit (AC22) with his +14 rapier:

In [ ]:
strike = pf2.check(14, DC=22)
pf2.outcome_counts(strike).to_pandas().to_frame("%") * 100.0

Or a wizard can blast the bandit, who has +10 reflex, with his DC21 fireball:

In [ ]:
reflex_save = pf2.check(10, DC=21)
pf2.outcome_counts(reflex_save).to_pandas().to_frame("%") * 100.0

Finally, with {func}`~pathfinder2e_stats.map_outcome` you can post-process the check outcome, for example to define the Evasion class feature or similar (if you roll a success, you get a critical success instead):

In [ ]:
save_with_evasion = pf2.map_outcome(reflex_save, evasion=True)
pf2.outcome_counts(save_with_evasion).to_pandas().to_frame("%") * 100.0

## Damage profiles

Previously, we saw how to roll raw 6d6. However, let's refine that - let's define the *damage profile* of a fireball, which we're going to roll in the next section.

In [ ]:
fireball = pf2.Damage("fire", 6, 6, basic_save=True)
fireball

Damage offers many keyword arguments and supports addition.
Let's have a rogue's 2d8+3 deadly d8 rapier, with 1d6 sneak attack:

In [ ]:
rapier = pf2.Damage("piercing", 2, 6, 3, deadly=8)
sneak_attack = pf2.Damage("precision", 1, 6)
rapier + sneak_attack

When rolling damage in the next chapter, we'll see that `pathfinder2e_stats` automatically manages the deadly, fatal, etc. traits.
To preview the breakdown of what's going to be rolled for each degree of success, we can call the {meth}`~pathfinder2e_stats.Damage.expand` method:

In [ ]:
(rapier + sneak_attack).expand()

Note how the `basic_save=True` flag on the fireball damage profile means it expands differently from a weapon:

In [ ]:
fireball.expand()

If basic save/basic attack damage rules, deadly, fatal, etc. are not enough, it's possible to hand-craft more sophisticated damage profiles with {class}`~pathfinder2e_stats.ExpandedDamage` - which is what you get when you call {meth}`~pathfinder2e_stats.Damage.expand`. You can define an {class}`~pathfinder2e_stats.ExpandedDamage` by initialising the class directly or by adding it to {class}`~pathfinder2e_stats.Damage`. For example, let's define a {prd_equipment}`Flaming <2838>` rune:

In [ ]:
flaming_rune = pf2.Damage("fire", 1, 6) + {
    pf2.DoS.critical_success: [pf2.Damage("fire", 1, 10, persistent=True)]
}
flaming_rune

Our rogue is getting an upgrade! Note that adding `Damage` + `ExpandedDamage` always expands the `Damage` first, so you'll no longer read *deadly d8* but the full success/critical success outcome for it.

In [ ]:
flaming_rapier = rapier + sneak_attack + flaming_rune
flaming_rapier

An {class}`~pathfinder2e_stats.ExpandedDamage` is just a fancy mapping of lists of {class}`~pathfinder2e_stats.Damage`, so usual mapping conversion techniques work:

In [ ]:
dict(flaming_rapier)

## Rolling damage

Now that we have the output of a {func}`~pathfinder2e_stats.check`, like an attack roll or a saving throw, and the {class}`~pathfinder2e_stats.Damage` profile, we can finally roll some {func}`~pathfinder2e_stats.damage`.

Let's reuse the strike outcome from above to roll damage for the flaming rapier:

In [ ]:
flaming_rapier_damage = pf2.damage(strike, flaming_rapier)
flaming_rapier_damage

{func}`~pathfinder2e_stats.damage` makes a copy of the dataset from the check outcome and adds variables to it. Again, most times we're going to care only about `total_damage`, but it can be interesting to understand how we got there:

- **direct_damage** how much immediate, simple damage we got on each of the 100,000 attacks. This is broken down by `damage_type` between piercing, precision and fire.
- **persistent_damage** persistent fire damage caused by the Flaming rune on critical hits. This is rolled by default for 3 rounds, after which we assume either that the target expired, the combat ended, or the persistent damage ended on its own.
- **persistent_damage_DC** DC for persistent damage to end on its own each round.
- **persistent_damage_check** the outcome of the flat check at the end of each of the 3 rounds to end the persistent damage from continuing into the next round.
- **apply_persistent_damage** whether the persistent_damage is still ongoing in this round or it already ended thanks to a successful save on a previous round.
- **total_damage** the sum of direct damage, persistent damage over all the rounds, and splash damage over multiple targets, with the damage type squashed.

{func}`~pathfinder2e_stats.damage` also supports defining weaknesses, resistances, and immunities.

We can invoke `.display` here too for a quick overview:

In [ ]:
flaming_rapier_damage.display(transpose=True)

From here we can start dicing and slicing with standard data science techniques. For example, let's plot the damage distribution:

In [ ]:
flaming_rapier_damage.total_damage.to_pandas().hist(
    bins=flaming_rapier_damage.total_damage.max().item() + 1
)

The above clearly shows the three distributions depending on the outcome of the attack roll:
- **Miss** and **Critical Miss** no damage
- **Hit** 4d6+3
- **Critical Hit** (4d6+3)x2 + 1d8 + 1d10 persistent over up to 3 rounds

Let's exclude misses:

In [ ]:
rapier_hit_dmg = flaming_rapier_damage.total_damage[
    flaming_rapier_damage.outcome >= pf2.DoS.success
]
rapier_hit_dmg.min().item()

In [ ]:
rapier_hit_dmg.to_pandas().hist(bins=rapier_hit_dmg.max().item())

Let's do the same for the fireball and let's observe the 4 intersecting distributions for the different saving throw outcomes:
- **Critical Success** no damage
- **Success** (6d6)/2
- **Failure** 6d6
- **Critical Failure** (6d6)x2

In [ ]:
fireball_damage = pf2.damage(reflex_save, fireball)
fireball_damage.total_damage.to_pandas().hist(
    bins=fireball_damage.total_damage.max().item() + 1
)

## Multiple targets and what-if analysis

You may want to hit multiple targets with the same fireball, each rolling a separate saving throw.

In almost all functions of `pathfinder2e_stats`, instead of scalars you can pass as inputs {class}`~xarray.DataArray` objects with arbitrary dimensions and coordinates.

Earlier, we had a bandit with +10 Reflex roll against a DC21 fireball:

In [ ]:
reflex_save = pf2.check(10, DC=21)

Let's have the fireball hit three targets instead:

In [ ]:
import xarray

reflex_bonus = xarray.DataArray(
    [10, 13, 11],
    dims=["target"],
    coords={"target": ["Alice", "Bob", "Charlie"]},
)
reflex_bonus

In the above example, the `coords` parameter is optional, but helps track what's what in the next steps.

We're going to roll the saving throw and the damage in a moment. Before we do that, we need to first go through the topic, in statistics, of *dependent* and *independent variables*. Two *dependent variables* are those that can be obtained by transforming the same original random variable through deterministic functions. Independent variables are those that don't; in other words they're not perfectly (anti)correlated.

In `pathfinder2e_stats`, every time you add extra dimensions to the inputs of {func}`~pathfinder2e_stats.check` or {func}`~pathfinder2e_stats.damage`, you need to declare whether the points along the new dimension should be rolled only once or separately, by passing parameters `independent_dims` and `dependent_dims`.

In the case of a fireball, each target rolls a separate check to save; the damage is rolled only once and then halved/doubled depending on the outcome:

In [ ]:
reflex_save = pf2.check(reflex_bonus, DC=21, independent_dims=["target"])
fireball_damage = pf2.damage(reflex_save, fireball, dependent_dims=["target"])
fireball_damage

Note how variables `natural`, `outcome`, `direct_damage` and `total_damage` have gained the extra dimension `target`, and how the `target` coordinate was propagated from the input. Let's have a look at some damage outputs to observe that saving throws were indeed independent but the damage is not:

In [ ]:
fireball_damage.total_damage.isel(roll=slice(10)).to_pandas()

Which dimensions are dependent and which are independent depends on the situation.
Consider:

- You want to hit 3 times with MAP. All attack and damage rolls are independent:

In [ ]:
MAP = xarray.DataArray([0, -5, -10], dims=["strike"])
strike = pf2.check(14 + MAP, DC=22, independent_dims=["strike"])
damage = pf2.damage(strike, flaming_rapier, independent_dims=["strike"])

- You want to hit two targets with {prd_feats}`Swipe <4795>`. You roll attack and damage only once; your attack may miss on one target and hit or critically hit on the other depending on their ACs:

In [ ]:
AC = xarray.DataArray([22, 15], dims=["target"])
strike = pf2.check(14, DC=AC, dependent_dims=["target"])
damage = pf2.damage(strike, flaming_rapier, dependent_dims=["target"])

- You want to know if it's worthwhile to spend an action to move into flanking position, given the same target. Roll attack and damage only once and compare it against the AC with and without flanking.
This type hypothetical study is commonly called a *what-if analysis*:

In [ ]:
off_guard = xarray.DataArray(
    [0, -2], dims=["off-guard"], coords={"off-guard": [False, True]}
)
strike = pf2.check(14, DC=22 + off_guard, dependent_dims=["off-guard"])
damage = pf2.damage(strike, flaming_rapier, dependent_dims=["off-guard"])

What's the impact on the damage distribution of the off-guard condition, given this specific attack bonus, base AC, and weapon?

In [ ]:
damage.total_damage.display()

You can combine extra dimensions arbitrarily. For example, let's strike the same target 3 times with increasing MAP, and observe how much getting them off-guard matters:

In [ ]:
strike = pf2.check(
    14 + MAP,
    DC=22 + off_guard,
    independent_dims=["strike"],
    dependent_dims=["off-guard"],
)
damage = pf2.damage(
    strike,
    flaming_rapier,
    independent_dims=["strike"],
    dependent_dims=["off-guard"],
)
damage

What's the mean damage of the three strikes?

In [ ]:
damage.total_damage.mean("roll").to_pandas()

Since labelling dimensions as (in)dependent on every `check` and `damage` call can get tedious, you can alternatively declare that, a dimension is always (in)dependent within your session:

In [ ]:
pf2.set_config(
    check_independent_dims=["strike"],
    check_dependent_dims=["off-guard"],
    damage_independent_dims=["strike"],
    damage_dependent_dims=["off-guard"],
)
strike = pf2.check(14 + MAP, DC=22 + off_guard)
damage = pf2.damage(strike, flaming_rapier)

## Conditional buffs/debuffs

Not everything in the game does damage; frequently it's worthwhile to spend an action to give to yourself or an ally a buff, or to debuff an enemy. A buff/debuff translates to a bonus/penalty to a check or to damage, or maybe to adding or removing a flat check (e.g. when it gives or removes the {prd_conditions}`Concealed <62>` or {prd_conditions}`Hidden <79>` conditions).

Crucially though, debuffs frequently involve a check that can negate or mitigate them. In `pathfinder2e_stats`, we can chain multiple actions, e.g. a {prd_spells}`Fear <1524>` spell followed by a Strike. On each of our 100,000 simulations, the AC of the target will change depending on the target's saving throw.

We're going to simulate this as follows:

1. Call `check` to roll a saving throw vs. Fear;
2. Map its outcome with {func}`~pathfinder2e_stats.map_outcome` to the severity of the Frightned condition;

In [ ]:
will_saving_throw = pf2.check(13, DC=21)
frightened = pf2.map_outcome(
    will_saving_throw.outcome,
    {
        pf2.DoS.critical_failure: 3,
        pf2.DoS.failure: 2,
        pf2.DoS.success: 1,
    },
)
frightened.display("frightened")

What is the probability to get each level of the Frightened condition from Fear?

In [ ]:
(
    frightened.value_counts("roll", new_dim="frightened", normalize=True)
    .to_pandas()
    .to_frame("%")
) * 100.0

3. Subtract the Frightened condition from the enemy's AC;
4. Roll `check` again for the Strike: the `DC` parameter will have a `roll` dimension;
5. Roll `damage` as usual.

In [ ]:
strike = pf2.check(14, DC=22 - frightened)
damage = pf2.damage(strike, flaming_rapier)

Note that we didn't need to clarify that `roll` is an independent dimension: `roll` is a special dimension that is always considered independent.
But wait! We wonder, was it worth the effort to cast Fear to begin with (assuming we don't care about anything but the extra damage on this Strike)? Let's do a *what-if analysis* for it:

In [ ]:
cast_fear = xarray.DataArray(
    [False, True], dims=["cast_fear"], coords={"cast_fear": [False, True]}
)
strike = pf2.check(14, DC=22 - frightened * cast_fear, dependent_dims=["cast_fear"])
damage = pf2.damage(strike, flaming_rapier, dependent_dims=["cast_fear"])
damage.total_damage.display()

In other words: what was the expected % bonus to damage from casting Fear, before the saving throw was rolled?

In [ ]:
mean_damage = damage.total_damage.mean("roll")
(mean_damage.sel(cast_fear=True) / mean_damage.sel(cast_fear=False)).item()

The Fear spell, _against this particular target and with this particular weapon_ (which, crucially, has the Deadly trait) gave us on average 20% extra damage on the Strike.

## Flat checks, disrupting actions and conditional actions

Some actions have a chance of failing. Among the most common causes, we have:
- targeting anything that is {prd_conditions}`Concealed <62>` or {prd_conditions}`Hidden <79>`;
- performing an action with the {prd_traits}`Manipulate <645>` trait (like casting most spells) while {prd_conditions}`Grabbed <77>`.

To model this, we need to introduce a special degree of success, `DoS.no_roll`. {func}`~pathfinder2e_stats.damage` always translates `DoS.no_roll` to zero damage.
It works as follows:
1. roll {func}`~pathfinder2e_stats.check` as normal;
2. roll another flat {func}`~pathfinder2e_stats.check` for the failure chance;
3. override the outcome of the first check with `DoS.no_roll` when the flat check failed;
4. roll {func}`~pathfinder2e_stats.damage` normally.

In [ ]:
strike = pf2.check(14, DC=22)
flat_check = pf2.check(
    DC=5, allow_critical_failure=False, allow_critical_success=False
)  # Concealed
strike["flat_check"] = flat_check.outcome
strike.outcome[strike.flat_check == pf2.DoS.failure] = pf2.DoS.no_roll
damage = pf2.damage(strike, flaming_rapier)

pf2.outcome_counts(strike).to_pandas().to_frame("%") * 100.0

Actions can also be disrupted; for example we may want to simulate the impact of receiving a {prd_actions}`Reactive Strike <2256>` while casting a spell and risk losing the spell on a critical hit.
It works in the same way as flat checks; you just update the check for the spell to `DoS.no_roll` as a function of the check for the reactive strike:

In [ ]:
reactive_strike = pf2.check(14, DC=22)
saving_throw = pf2.check(reflex_bonus, DC=21, independent_dims=["target"])
saving_throw["disrupted"] = reactive_strike.outcome == pf2.DoS.critical_success
saving_throw.outcome[saving_throw.disrupted] = pf2.DoS.no_roll
fireball_damage = pf2.damage(saving_throw, fireball, dependent_dims=["target"])

In [ ]:
saving_throw.disrupted.value_counts(
    "roll", new_dim="disrupted", normalize=True
).to_pandas().to_frame("%") * 100.0

Test that the fireball never did any damage whenever it was disrupted:

In [ ]:
fireball_damage.total_damage[fireball_damage.disrupted].max().item()

A player or the GM may also completely change course of action depending on the outcome of a roll.
For example, one may be {prd_conditions}`Restrained <90>` and decide that they are going to try to {prd_actions}`Escape <2296>`; if successful they'll cast a spell otherwise they'll try again, unless it's a critical failure:

In [ ]:
# Try up to 3 times per round...
escape = pf2.check(8 + MAP, DC=22)
# ..but a critical failure failure stops you from retrying this round.
# We're retrying exclusively when the previous attempt was exactly a failure.
for i in (1, 2):
    escape.outcome.isel(strike=i)[
        escape.outcome.isel(strike=i - 1) != pf2.DoS.failure
    ] = pf2.DoS.no_roll

In [ ]:
pf2.outcome_counts(escape).to_pandas() * 100.0

What's the probability of escaping by the end of the round?

In [ ]:
(
    (escape.outcome >= pf2.DoS.success)
    .any("strike")
    .value_counts(dim="roll", new_dim="escaped", normalize=True)
    .to_pandas()
    .to_frame("%")
) * 100.0

Now let's cast the spell, but only if we escaped on the first round:

In [ ]:
can_cast = escape.isel(strike=0).outcome >= pf2.DoS.success
reflex_save = pf2.check(10, DC=21)
reflex_save.outcome[~can_cast] = pf2.DoS.no_roll
pf2.outcome_counts(reflex_save).to_pandas().to_frame("%") * 100.0

## Armory and tables

For the sake of convenience, we don't need to write by hand the damage spec of the *+1 Striking Flaming Rapier* from above every time. {doc}`pf2.armory <../armory>` offers a wealth of weapons, runes, spells, and common class features:

In [ ]:
(
    pf2.armory.swords.rapier(dice=2)
    + pf2.armory.runes.flaming()
    + pf2.armory.class_features.sneak_attack(level=5)
)

We don't need to calculate our character's attack bonus either. {doc}`pf2.tables.PC <tables>` offers a wealth of precalculated progressions over 20 levels for most optimized character builds:

In [ ]:
pf2.tables.PC

Each table has a `level` dimension, plus variables and extra dimensions depending on the table:

In [ ]:
pf2.tables.PC.weapon_proficiency.display()

We can build the attack bonus of the rogue from earlier by picking what we want from the PC tables:

In [ ]:
rogue_atk_bonus = (
    # Start with DEX+4 at level 1 and always increase it
    pf2.tables.PC.ability_bonus.boosts.sel(initial=4, drop=True)
    # Get an Apex item at level 17 for +1 DEX
    + pf2.tables.PC.ability_bonus.apex
    # Upgrade weapons as soon as possible: +1 at level 2, +2 at level 10, etc.
    + pf2.tables.PC.attack_item_bonus.potency_rune
    # Trained (+2) at level 1, Expert (+4) at level 5, Master (+6) at level 13
    + pf2.tables.PC.weapon_proficiency.rogue
    # Add level to proficiency
    + pf2.tables.PC.level
)
rogue_atk_bonus.display(transpose=True)

Since adding up all of the above is tedious and error prone, {doc}`pf2.tables.SIMPLE_PC <tables>` is a simplified variant that makes a bunch of (opinable) choices, giving you an even more standardized progression:

In [ ]:
rogue_atk_bonus = pf2.tables.SIMPLE_PC.weapon_attack_bonus.rogue
rogue_atk_bonus.display()

So our level 5 rogue will have an attack bonus of

In [ ]:
rogue_atk_bonus.sum("component").sel(level=5).item()

Note that the above is just a *typical baseline*, and does not take into consideration buffs, debuffs, suboptimal equipment, or uncommon character progression choices.

There are more {doc}`tables` available:

- `pf2.tables.DC` contains the {prd_rules}`Difficulty Classes <2627>` tables from the GM Core;
- `pf2.tables.EARN_INCOME` is the {prd_actions}`Earn Income <2364>` table from the Player Core;
- `pf2.tables.NPC` gives you the tables from the {prd_rules}`Building Creatures <2874>` chapter of the GM Core;
- `pf2.tables.SIMPLE_NPC` gives you a simplified version of `NPC` with just three targets to blast
  with your attack and spells (or to get blasted by):
  - a weak minion of your level - 2 with all stats rated Low;
  - a worthy foe of your level with all stats rated Moderate; and
  - a boss of your level + 2 with all stats rated High.

In [ ]:
# One very easy, one average and one very hard enemy at level 5
pf2.tables.SIMPLE_NPC.sel(level=5).display(transpose=True)

## Next steps

Congratulations, you finished the basic tutorial!

From here, you can go look at the {doc}`index`.
In the {doc}`../api`, you will find many functions, flags and options that were omitted here for the sake of brevity.